In [0]:
root = "abfss://imoveis@datalakepriscylla.dfs.core.windows.net"
display(dbutils.fs.ls(root))  # listar
test = f"{root}/bronze/_uc_smoketest/"
spark.range(3).write.mode("overwrite").parquet(test)
display(dbutils.fs.ls(test))
dbutils.fs.rm(test, True)


In [0]:
# 0) ajuste se necessário
root = "abfss://imoveis@datalakepriscylla.dfs.core.windows.net"

# 1) listar o que há em inbound/
display(dbutils.fs.ls(f"{root}/inbound"))

# 2) procurar um arquivo (procura recursivo até 5 níveis)
def list_files_recursive(base, max_depth=5):
    stack = [(base.rstrip("/"), 0)]
    files = []
    while stack:
        cur, d = stack.pop()
        for fi in dbutils.fs.ls(cur + "/"):
            # diretório termina com "/"
            if fi.name.endswith("/") and d < max_depth:
                stack.append((fi.path.rstrip("/"), d + 1))
            else:
                files.append(fi.path)
    return files

files = list_files_recursive(f"{root}/inbound")
print("Encontrados", len(files), "arquivos (recursivo). Exemplo:", files[:5])

# 3) escolher o primeiro parquet/csv/json e ler
picked = None
for ext in (".parquet", ".csv", ".json"):
    for p in files:
        if p.lower().endswith(ext):
            picked = (ext, p); break
    if picked: break

if not picked:
    print("⚠️ Não encontrei parquet/csv/json dentro de inbound/. Verifique os nomes/extensões ou me diga o formato.")
else:
    ext, path = picked
    print("Vou ler:", path)
    if ext == ".parquet":
        df = spark.read.parquet(path)      # se for dataset, também funciona apontar para a pasta
    elif ext == ".csv":
        df = spark.read.option("header","true").csv(path)
    else:  # .json
        df = spark.read.json(path)

    print("✅ Leitura OK — linhas:", df.count())
    display(df.limit(20))


In [0]:
path = "abfss://imoveis@datalakepriscylla.dfs.core.windows.net/inbound/dados_brutos_imoveis.json"
df = spark.read.json(path)
display(df.limit(20))

In [0]:
df = df.drop("imagens", "usuario")

In [0]:
display(df.limit(20))

In [0]:
from pyspark.sql.functions import col

df = df.withColumn("id", col("anuncio.id"))
display(df)

In [0]:
df.write.format("delta").mode("overwrite").save(f"{root}/bronze/dados_brutos_imoveis")